In [56]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [65]:
import tensorflow as tf

class TensorFlowModel: 
    def __init__(self, inputs,targets, number_of_neurons, learning_rate, batch_size, activation, initial_weights) -> None:
        self.inputs = tf.constant(inputs,dtype=tf.float32)
        self.targets = tf.constant(targets, dtype=tf.float32)
        self.initial_weights = tf.constant(initial_weights, dtype=tf.float32)
        self.input_size = len(inputs[0])
        self.number_of_neurons = number_of_neurons 
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.dataset = tf.data.Dataset.from_tensor_slices((self.inputs, self.targets))
        self.dataset = self.dataset.batch(batch_size)  # Batch the data
        self.initial_bias = tf.ones([number_of_neurons], dtype=tf.float32)
        self.activation = activation
        self.model = tf.keras.Sequential([
                        tf.keras.layers.Dense(number_of_neurons,
                                            input_shape=(self.input_size,),
                                            activation=self.activation,
                                            use_bias=True)
                    ])
        self.model.layers[0].set_weights([self.initial_weights, self.initial_bias])
    
    def train_step(self,inputs, target):
        optimizer = tf.keras.optimizers.SGD(learning_rate=self.learning_rate)
        loss_function = tf.keras.losses.MeanSquaredError()
        with tf.GradientTape() as tape:
            predictions = self.model(inputs, training=True)
            loss = loss_function(self.targets, predictions)
        # Only update the weights (first variable in trainable_variables)
        weights = self.model.trainable_variables[0]  # This assumes the first variable is the weight matrix
        gradients = tape.gradient(loss, [weights])
        optimizer.apply_gradients(zip(gradients, [weights]))
        return loss

    def show_prediction(self):
        print("========================================")
        updated_weights = self.model.layers[0].get_weights()[0]
        print("Updated weights:\n", updated_weights)
        print("=========================================")

    def predict(self,max_iteration):
        for i in range(max_iteration):
            # Training loop
            for inputs_batch, targets_batch in self.dataset:
                loss = self.train_step(inputs_batch, targets_batch)
        self.show_prediction()


In [66]:
inputs = [[1.0, 3.0, 1.0], [1.0, 1.0, 2.0]]
targets = [[2.0, 0.3, -1.9], [1.3, -0.7, 0.1]]
number_of_neurons = 3
learning_rate = 0.001
batch_size = 2
activation = "linear" 
initial_weights = [
              [0.1, 0.3, 0.2],
              [0.4, 0.2, -0.7],
              [0.1, -0.8, 0.5]
            ]
max_iteration = 3
model = TensorFlowModel(inputs, targets, number_of_neurons, learning_rate, batch_size, activation, initial_weights)
model.predict(3)

Updated weights:
 [[ 0.09920387  0.29860693  0.19711408]
 [ 0.39840838  0.19701517 -0.7058694 ]
 [ 0.09880546 -0.8019903   0.49571985]]


c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
